In [1]:
import pickle
from attention import *
from torch.utils.data import DataLoader
from pytorch_util import RAdam,trainable_parameter
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [9]:
batch_size = 2048
dim = 512
head_d = 8
head = SimplyInteraction #SimplyHead
encoder_layer,decoder_layer = 2,2
epochs = 10
clip = 0.5
logLoss = True

dim_feedforward = 1024
dropout = 0.1
lr = 1e-1

In [3]:
with open('../Data/train_data_attention_node.pickle', 'rb') as handle:
    train_node = pickle.load(handle)
with open('../Data/train_data_attention_edge.pickle', 'rb') as handle:
    train_edge = pickle.load(handle)
with open('../Data/train_data_attention_edge_y.pickle', 'rb') as handle:
    train_y = pickle.load(handle)

In [4]:
train_dl = attentionDataset(train_node[:4000000],train_edge[:4000000],train_y[:4000000])
train_dl = DataLoader(train_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn,num_workers=4)
val_dl = attentionDataset(train_node[4000000:],train_edge[4000000:],train_y[4000000:])
val_dl = DataLoader(val_dl,shuffle=False,batch_size=batch_size,collate_fn=collate_fn,num_workers=4)

In [10]:
model=Attention(dim,encoder_layer,decoder_layer,head_d,\
                head,dropout=dropout,dim_feedforward=dim_feedforward).to('cuda')
paras = trainable_parameter(model)
opt = RAdam(paras,lr=lr,weight_decay=1e-2)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-04)

In [11]:
# out,mask,edge,y = next(iter(train_dl))
# out,mask,edge,y = out.to('cuda:0'),mask.to('cuda:0'),edge.to('cuda:0'),y.to('cuda:0')

# opt.zero_grad()
# loss,loss_perType = model(out,mask,edge,y,logLoss)
# loss.backward()

# [(n,p.std()) for n,p in model.named_parameters()]

# [(n,p.grad.mean()) for n,p in model.named_parameters()]

In [12]:
model,bestOpt,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

epoch:0, train_loss: +1.656, val_loss: +2.735, 
train_vector: +2.70|+2.51|+1.42|+1.48|+1.37|+1.32|+1.44|+0.99, 
val_vector  : +3.44|+1.78|+2.67|+2.65|+2.60|+2.79|+2.68|+2.50

epoch:1, train_loss: +1.978, val_loss: +4.704, 
train_vector: +2.71|+2.60|+1.81|+1.85|+1.75|+1.74|+1.81|+1.56, 
val_vector  : +4.83|+3.95|+4.70|+4.68|+4.44|+4.71|+4.57|+4.47

epoch:2, train_loss: +1.752, val_loss: +4.407, 
train_vector: +2.59|+2.50|+1.55|+1.60|+1.50|+1.47|+1.56|+1.25, 
val_vector  : +4.33|+4.28|+4.42|+4.21|+4.14|+4.41|+4.15|+4.14

epoch:3, train_loss: +1.653, val_loss: +5.002, 
train_vector: +2.54|+2.46|+1.44|+1.49|+1.38|+1.35|+1.45|+1.12, 
val_vector  : +6.22|+4.27|+4.78|+4.68|+4.52|+4.81|+4.79|+4.56

epoch:4, train_loss: +1.668, val_loss: +4.967, 
train_vector: +2.55|+2.46|+1.45|+1.50|+1.40|+1.36|+1.47|+1.14, 
val_vector  : +4.98|+4.18|+4.98|+4.93|+4.67|+4.97|+4.94|+4.71

epoch:5, train_loss: +1.665, val_loss: +5.861, 
train_vector: +2.54|+2.45|+1.45|+1.50|+1.40|+1.36|+1.47|+1.14, 
val_vector  :